In [24]:
import spacy
import en_core_web_sm
import benepar
from typing import List
from spacy.matcher import Matcher
from concept_processing.nlp.spacy_wrapper import SpacyWrapper

In [7]:
# benepar.download('benepar_en3')

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /Users/Cherry0904/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


True

In [58]:
def post_process_atomisation(atomic_sents: List[str]) -> List[str]:
    # nlp = SpacyWrapper()
    nlp = en_core_web_sm.load()
    excmatcher = add_exc_matcher(nlp)

    num_of_words = [len(sent.split()) for sent in atomic_sents]
    index_of_short_concepts = [i for i, j in enumerate(num_of_words) if j == 2 or j==3]
    index_of_all_concepts = [i for i, j in enumerate(num_of_words)]
    index_of_long_concepts = [x for x in index_of_all_concepts if x not in index_of_short_concepts]
    # index_of_long_concepts = [i for i, j in enumerate(num_of_words) if j != 2 and j !=3]
    print(index_of_long_concepts)
    # index_of_3_token_concepts = [i for i, j in enumerate(num_of_words) if j == 3]
        
    index_to_keep = []
    for i in index_of_short_concepts:
        doc = nlp(atomic_sents[i])
        match = excmatcher(doc)

        if match == []: # If there is no match
            index_to_keep.append(i)

    atomic_sents = [atomic_sents[i] for i in index_to_keep + index_of_long_concepts]

    return atomic_sents



def add_exc_matcher(nlp):
    # create exclusion matcher for our concepts
    excpattern1 = [{"POS": {"IN": ["NOUN", "PRON"]}}, {"POS": "VERB"}] # such as "it looks"
    # excpattern2 = [{"POS": "DET"}, {"POS": "NOUN"}, {"POS": "VERB"}] # such as "the woman looks"
        
    # Dirty way of using SpacyWrapper, kept because this code is not maintained
    # excmatcher = Matcher(nlp._nlp.vocab)
    excmatcher = Matcher(nlp.vocab)

    excmatcher.add("short_concept_2_tokens", [excpattern1])
    # excmatcher.add("short_concept_3_tokens", [excpattern2])
    # nlp._nlp.add_pipe("benepar", config={"model": "benepar_en3"})

    return excmatcher

In [60]:
atomic_sents = ["It looks.", "She looks.", "Me laugh.", "The people look.", "The colors make.", "Happy birthday.", "Brilliant.", "Love you."]
post_processed_atomic_sents = post_process_atomisation(atomic_sents)
print(post_processed_atomic_sents)

/Users/Cherry0904/Desktop/roko-for-charlize/venv/lib/python3.8/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[6]
['Happy birthday.', 'Love you.', 'Brilliant.']


In [47]:
nlp = en_core_web_sm.load()
excmatcher = add_exc_matcher(nlp)

atomic_sents = ["Happy."]
doc = nlp(atomic_sents[0])
match = excmatcher(doc)
print(match)
# for match_id, start, end in match:
#     span = doc[start:end]
#     print(start, end, span)


[]


In [38]:
for match_id, start, end in match:
    span = doc[start:end]
    print(start, end, span)